In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikeras

In [ ]:
!pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install -U nltk

In [ ]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from keras.layers import Dense, Dropout
from keras.models import Model, Sequential

from keras.layers import Bidirectional
import keras.layers as layers
from keras.models import Sequential

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV

In [ ]:
from os import listdir
from os.path import isfile, join, isdir
import re
import pickle
import tensorflow as tf
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from keras.callbacks import EarlyStopping
from sklearn.metrics import make_scorer

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from drive.MyDrive.aathesis.func import *
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [ ]:
nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_df = text_to_df('./drive/MyDrive/aathesis/increment_datasets_f2/pc/orig_full.txt')
data_df = data_df[:2500]

In [ ]:
def clean_text(text):
  text = text.lower()
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = BAD_SYMBOLS_RE.sub('', text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS)
  text = stemmer.stem(text)
  return text

In [ ]:
data_df['text'] = data_df['text'].apply(clean_text)

In [ ]:
data_df.to_csv("./drive/MyDrive/aathesis/data_clean.txt", sep='\t', index = None, header = None)

In [ ]:
test_df = text_to_df('/content/drive/MyDrive/aathesis/increment_datasets_f2/pc/test.txt')
test_df['text'] = test_df['text'].apply(clean_text)

In [ ]:
data_df = text_to_df('/content/drive/MyDrive/aathesis/data_clean.txt')
test_df = text_to_df('/content/drive/MyDrive/aathesis/test_clean.txt')

In [ ]:
data_df.drop(data_df[data_df['text'].isna() == True].index, inplace = True)
test_df.drop(test_df[test_df['text'].isna() == True].index, inplace = True)

In [ ]:
from keras.preprocessing.text import Tokenizer
from sklearn.base import BaseEstimator, TransformerMixin

class TokenizerTransformer(BaseEstimator, TransformerMixin, Tokenizer):

    def __init__(self, **tokenizer_params):
        Tokenizer.__init__(self, **tokenizer_params)

    def fit(self, X, y=None):
        self.fit_on_texts(X)
        return self

    def transform(self, X, y=None):
        X_transformed = self.texts_to_sequences(X)
        return X_transformed

In [ ]:
from keras.preprocessing.sequence import pad_sequences


class PadSequencesTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, maxlen):
        self.maxlen = maxlen

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_padded = pad_sequences(X, maxlen=self.maxlen)
        return X_padded

## Load GloVe Embedding

In [ ]:
!unzip '/content/drive/MyDrive/Copy of idwiki_word2vec_300.zip'

Archive:  /content/drive/MyDrive/Copy of idwiki_word2vec_300.zip
  inflating: idwiki_word2vec_300.model.trainables.syn1neg.npy  
  inflating: idwiki_word2vec_300.model.wv.vectors.npy  
  inflating: idwiki_word2vec_300.model  


In [ ]:
wv = KeyedVectors.load("/content/idwiki_word2vec_300.model")
word_vectors = wv.wv
word_vector = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [ ]:
glove_file = datapath('/content/drive/MyDrive/aathesis/glove.6B.300d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.300d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)

<ipython-input-16-2b0ec80d785e>:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_glove_file)


(400000, 300)

In [ ]:
word_vector = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [ ]:
vocab_size = len(word_vectors) + 1
word2vec_len = 300
#longest_text = data_df['text'].str.split().str.len().max()
sentence_length = 30

In [ ]:
# create a weight matrix for all words in training text --> size (1800, 300)
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
  embedding_vector = word_vectors.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
    print(word)

In [ ]:
embedding_weights = np.vstack([
           np.zeros(word_vectors.vectors.shape[1]),
           word_vectors.vectors
])

In [ ]:
models = list()
resample = list()
acc_list = list()
precision_list = list()
recall_list = list()
train_length = list()
test_length = list()

In [ ]:
def test_evals(clf_model, X_test, y_test, train_len, algo=None, sampling=None):
  # Test set prediction
  #y_prob=clf_model.predict_proba(X_test)
  y_pred=clf_model.predict(X_test)
  print('Confusion Matrix')
  print('='*60)
  print(confusion_matrix(y_test,y_pred),"\n")
  print('Classification Report')
  print('='*60)
  print(classification_report(y_test,y_pred),"\n")

  models.append(algo)
  acc_list.append(accuracy_score(y_test,y_pred))
  precision_list.append(precision_score(y_test,y_pred))
  recall_list.append(recall_score(y_test,y_pred))
  resample.append(sampling)
  train_length.append(train_len)
  test_length.append(len(X_test))

In [ ]:
def evals(clf_model, X_test, y_test, train_len, algo=None, sampling=None):
  # Test set prediction
  loss, accuracy,precision,recall = clf_model.evaluate(X_test,y_test)

  models.append(algo)
  acc_list.append(accuracy)
  precision_list.append(precision)
  recall_list.append(recall)
  resample.append(sampling)
  train_length.append(train_len)
  test_length.append(len(X_test))

In [ ]:
my_tokenizer = TokenizerTransformer()
my_padder = PadSequencesTransformer(maxlen=30)
my_tokenizer.fit(data_df['text'])
tokenized = my_tokenizer.transform(data_df['text'])
my_padder.fit(tokenized)
X_train = my_padder.transform(tokenized)
Y_train = data_df['label']

test_tokenizer = TokenizerTransformer()
test_padder = PadSequencesTransformer(maxlen=30)
test_tokenizer.fit(test_df['text'])
test_tokenized = test_tokenizer.transform(test_df['text'])
test_padder.fit(test_tokenized)
X_test = test_padder.transform(test_tokenized)
Y_test = test_df['label']

## XGBoost

In [ ]:
xgb_model = XGBClassifier(n_jobs=-1)

eval_set = [(X_test, Y_test)]
#train model
xgb_model.fit(X_train,
      Y_train,
      verbose=True)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
test_evals(xgb_model, X_test, Y_test, len(data_df['text']), 'XGBoost', 'No Aug')

Confusion Matrix
[[197  99]
 [143  61]] 

Classification Report
              precision    recall  f1-score   support

           0       0.58      0.67      0.62       296
           1       0.38      0.30      0.34       204

    accuracy                           0.52       500
   macro avg       0.48      0.48      0.48       500
weighted avg       0.50      0.52      0.50       500
 



In [ ]:
smote = SMOTE(random_state=42)
sm_train_x, sm_train_y = smote.fit_resample(X_train, Y_train)
xgb_model.fit(sm_train_x, sm_train_y)
test_evals(xgb_model, X_test, Y_test, len(sm_train_x), 'XGBoost', 'SMOTE')

Confusion Matrix
[[160 136]
 [110  94]] 

Classification Report
              precision    recall  f1-score   support

           0       0.59      0.54      0.57       296
           1       0.41      0.46      0.43       204

    accuracy                           0.51       500
   macro avg       0.50      0.50      0.50       500
weighted avg       0.52      0.51      0.51       500
 



In [ ]:
adasyn = ADASYN(random_state=42)
ada_train_x, ada_train_y = adasyn.fit_resample(X_train, Y_train)
xgb_model.fit(ada_train_x, ada_train_y)
test_evals(xgb_model, X_test, Y_test, len(ada_train_x), 'XGBoost', 'ADASYN')

Confusion Matrix
[[157 139]
 [110  94]] 

Classification Report
              precision    recall  f1-score   support

           0       0.59      0.53      0.56       296
           1       0.40      0.46      0.43       204

    accuracy                           0.50       500
   macro avg       0.50      0.50      0.49       500
weighted avg       0.51      0.50      0.51       500
 



In [ ]:
smtomek = SMOTETomek(random_state=42)
tom_train_x, tom_train_y = smtomek.fit_resample(X_train, Y_train)
xgb_model.fit(tom_train_x, tom_train_y)
test_evals(xgb_model, X_test, Y_test, len(tom_train_x), 'XGBoost', 'SMOTE + Tomek')

Confusion Matrix
[[163 133]
 [108  96]] 

Classification Report
              precision    recall  f1-score   support

           0       0.60      0.55      0.57       296
           1       0.42      0.47      0.44       204

    accuracy                           0.52       500
   macro avg       0.51      0.51      0.51       500
weighted avg       0.53      0.52      0.52       500
 



In [ ]:
sm_enn = SMOTEENN(random_state=42)
enn_train_x, enn_train_y = sm_enn.fit_resample(X_train, Y_train)
xgb_model.fit(enn_train_x, enn_train_y)

test_evals(xgb_model, X_test, Y_test, len(enn_train_x), 'XGBoost', 'SMOTE + ENN')

Confusion Matrix
[[122 174]
 [ 82 122]] 

Classification Report
              precision    recall  f1-score   support

           0       0.60      0.41      0.49       296
           1       0.41      0.60      0.49       204

    accuracy                           0.49       500
   macro avg       0.51      0.51      0.49       500
weighted avg       0.52      0.49      0.49       500
 



In [ ]:
rus = RandomUnderSampler(random_state=42)
rus_train_x, rus_train_y = rus.fit_resample(X_train, Y_train)
xgb_model.fit(rus_train_x, rus_train_y)

test_evals(xgb_model, X_test, Y_test, len(rus_train_y), 'XGBoost', 'Random Undersampling')

Confusion Matrix
[[129 167]
 [ 91 113]] 

Classification Report
              precision    recall  f1-score   support

           0       0.59      0.44      0.50       296
           1       0.40      0.55      0.47       204

    accuracy                           0.48       500
   macro avg       0.49      0.49      0.48       500
weighted avg       0.51      0.48      0.49       500
 



In [ ]:
ros = RandomOverSampler(random_state=42)
ros_train_x, ros_train_y = ros.fit_resample(X_train, Y_train)
xgb_model.fit(ros_train_x, ros_train_y)

test_evals(xgb_model, X_test, Y_test, len(ros_train_y), 'XGBoost', 'Random Oversampling')

Confusion Matrix
[[198  98]
 [131  73]] 

Classification Report
              precision    recall  f1-score   support

           0       0.60      0.67      0.63       296
           1       0.43      0.36      0.39       204

    accuracy                           0.54       500
   macro avg       0.51      0.51      0.51       500
weighted avg       0.53      0.54      0.53       500
 



## AdaBoost

In [ ]:
adaboost_model = AdaBoostClassifier(n_estimators=50, learning_rate=1)

eval_set = [(X_test, Y_test)]
#train model
adaboost_model.fit(X_train,
      Y_train,
      verbose=True)
test_evals(adaboost_model, X_test, Y_test, len(data_df['text']), 'AdaBoost', 'No Aug')

In [ ]:
# smote = SMOTE(random_state=42)
# sm_train_x, sm_train_y = smote.fit_resample(X_train, Y_train)
adaboost_model.fit(sm_train_x, sm_train_y)
test_evals(adaboost_model, X_test, Y_test, len(sm_train_x), 'AdaBoost', 'SMOTE')

In [ ]:
# adasyn = ADASYN(random_state=42)
# ada_train_x, ada_train_y = adasyn.fit_resample(X_train, Y_train)
adaboost_model.fit(ada_train_x, ada_train_y)
test_evals(adaboost_model, X_test, Y_test, len(ada_train_x), 'AdaBoost', 'ADASYN')

In [ ]:
# smtomek = SMOTETomek(random_state=42)
# tom_train_x, tom_train_y = smtomek.fit_resample(X_train, Y_train)
adaboost_model.fit(tom_train_x, tom_train_y)
test_evals(adaboost_model, X_test, Y_test, len(tom_train_x), 'AdaBoost', 'SMOTE + Tomek')

In [ ]:
# sm_enn = SMOTEENN(random_state=42)
# enn_train_x, enn_train_y = sm_enn.fit_resample(X_train, Y_train)
adaboost_model.fit(enn_train_x, enn_train_y)

test_evals(adaboost_model, X_test, Y_test, len(enn_train_x), 'AdaBoost', 'SMOTE + ENN')

In [ ]:
# rus = RandomUnderSampler(random_state=42)
# rus_train_x, rus_train_y = rus.fit_resample(X_train, Y_train)
adaboost_model.fit(rus_train_x, rus_train_y)

test_evals(adaboost_model, X_test, Y_test, len(rus_train_y), 'AdaBoost', 'Random Undersampling')

In [ ]:
# ros = RandomOverSampler(random_state=42)
# ros_train_x, ros_train_y = ros.fit_resample(X_train, Y_train)
adaboost_model.fit(ros_train_x, ros_train_y)

test_evals(adaboost_model, X_test, Y_test, len(ros_train_y), 'AdaBoost', 'Random Oversampling')

## CNN

In [ ]:
#building the cnn in keras
def build_cnn(sentence_length, word2vec_len):
  model = Sequential([
     Embedding(vocab_size, word2vec_len, input_length=sentence_length),
     Conv1D(128, 5, activation='relu'),
     GlobalMaxPooling1D(),
     Dense(20, activation='relu'),
     Dense(1, kernel_initializer='normal', activation='sigmoid')
     ])
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  return model

In [ ]:
params = {
}
scorers = {
        'precision_score': make_scorer(precision_score),
        'recall_score': make_scorer(recall_score),
        'accuracy_score': make_scorer(accuracy_score)
        }
cnn_model = build_cnn(sentence_length, word2vec_len)
# cnn_m = KerasClassifier(model = cnn_model, optimizer="adam", epochs=5, verbose=0)
# search = GridSearchCV(estimator=cnn_m, param_grid=params, cv=5, scoring = scorers,
#                       refit='accuracy_score', return_train_score=True)
cnn_model.fit(X_train, Y_train, epochs=5, verbose=True, shuffle=False)

Epoch 1/5
79/79 [==============================] - 13s 147ms/step - loss: 0.5576 - accuracy: 0.7175 - precision_6: 0.6375 - recall_6: 0.1360
Epoch 2/5
79/79 [==============================] - 2s 25ms/step - loss: 0.3077 - accuracy: 0.8808 - precision_6: 0.8096 - recall_6: 0.7880
Epoch 3/5
79/79 [==============================] - 2s 26ms/step - loss: 0.1173 - accuracy: 0.9532 - precision_6: 0.9451 - recall_6: 0.8960
Epoch 4/5
79/79 [==============================] - 2s 26ms/step - loss: 0.0353 - accuracy: 0.9872 - precision_6: 0.9825 - recall_6: 0.9747
Epoch 5/5
79/79 [==============================] - 2s 28ms/step - loss: 0.0170 - accuracy: 0.9924 - precision_6: 0.9854 - recall_6: 0.9893


In [ ]:
#predictions = cnn_model.predict(X_test)
#accuracy_score(Y_test, predictions)
evals(cnn_model, X_test, Y_test, len(X_test), 'CNN', 'No Aug')

16/16 [==============================] - 0s 6ms/step - loss: 2.6972 - accuracy: 0.5580 - precision_6: 0.3924 - recall_6: 0.1520


In [ ]:
clf_eval_df = pd.DataFrame({'model':models,
                            'resample':resample,
                            'accuracy': acc_list,
                            'precision':precision_list,
                            'recall':recall_list,
                            'train length':train_length,
                            'test_length':test_length})

In [ ]:
clf_eval_df

,model,resample,accuracy,precision,recall,train length,test_length
0,XGBoost,No Aug,0.516,0.381250,0.299020,2499,500
1,XGBoost,SMOTE,0.508,0.408696,0.460784,3498,500
2,XGBoost,ADASYN,0.502,0.403433,0.460784,3445,500
3,XGBoost,SMOTE + Tomek,0.518,0.419214,0.470588,3392,500
4,XGBoost,SMOTE + ENN,0.488,0.412162,0.598039,1714,500
5,XGBoost,Random Undersampling,0.484,0.403571,0.553922,1500,500
6,XGBoost,Random Oversampling,0.542,0.426901,0.357843,3498,500
7,Bi-LSTM,SMOTE + ENN,0.592,0.000000,0.000000,1631,500
8,Bi-LSTM,No Aug,0.746,0.750503,0.746000,2500,500
9,Bi-LSTM,No Aug,0.996,0.000000,0.000000,2499,500


## BiLSTM

In [ ]:
def get_x_y(df, num_classes, word2vec_len, input_size, word2vec):

  #read in lines
  train_lines = df
  num_lines = len(train_lines)

  #initialize x and y matrix
  x_matrix = None
  y_matrix = None

  try:
    x_matrix = np.zeros((num_lines, input_size, word2vec_len))
  except:
    print("Error!", num_lines, input_size, word2vec_len)
  y_matrix = np.zeros((num_lines, num_classes))

  #insert values
  for i, line in enumerate(train_lines):
    #parts = line[:-1].split('\t')
    label = int(train_lines['label'][i])
    sentence = train_lines['text'][i]

    #insert x
    words = sentence.split(' ')
    words = words[:x_matrix.shape[1]] #cut off if too long
    for j, word in enumerate(words):
      if word in word2vec:
        x_matrix[i, j, :] = word2vec[word]

    #insert y
    y_matrix[i][label] = 1.0
  return x_matrix, y_matrix

In [ ]:
def build_models(sentence_length, word2vec_len):
	model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(sentence_length, word2vec_len)),
    Dropout(0.5),
    Bidirectional(LSTM(32, return_sequences=False)),
    Dropout(0.5),
    Dense(20, activation='relu'),
    Dense(2, activation='sigmoid')
    ])
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
	#print(model.summary())
	return model

In [ ]:
model_lstm = build_models(sentence_length, word2vec_len)

In [ ]:
#load data
# data_train = '/content/drive/MyDrive/aathesis/data_clean.txt'
# data_test = '/content/drive/MyDrive/aathesis/test_clean.txt'
train_x, train_y = get_x_y(data_df, 2, word2vec_len, sentence_length, word_vectors)
test_x, test_y = get_x_y(test_df, 2, word2vec_len, sentence_length, word_vectors)

#implement early stopping
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

#train model
model_lstm.fit(train_x, train_y, epochs=100, callbacks=callbacks,
               validation_split=0.1,
               batch_size=32,
               shuffle=False,
               verbose=1)

In [ ]:
evals(model_lstm, test_x, test_y, len(train_x), 'Bi-LSTM', 'No Aug')

16/16 [==============================] - 0s 7ms/step - loss: 0.0320 - accuracy: 0.9960 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00


In [ ]:
#evaluate model
y_preds = model_lstm.predict(test_x)
test_y_cat = one_hot_to_categorical(test_y)
y_pred_cat = one_hot_to_categorical(y_preds)
acc = accuracy_score(test_y_cat, y_pred_cat)

16/16 [==============================] - 2s 7ms/step


In [ ]:
acc

0.996

In [ ]:
reshaped_train = np.array(train_x).reshape(train_x.shape[0], train_x.shape[1] * train_x.shape[2])
#reshaped_test = np.array(enn_train_x).reshape(X_test.shape[0], X_test.shape[1], word2vec_len)

In [ ]:
reshaped_train.shape

(2499, 9000)

In [ ]:
reshaped_train = train_x[:, :, 0]

In [ ]:
result = train_x[:, :, 0]
result.shape

(2499, 30)

In [ ]:
result[0], X_train[0]

(array([-0.83695924, -0.58585596,  0.85513383,  0.29589596,  0.65106684,
         0.25178507,  1.57285666, -0.35270053,  0.00830237, -0.13255566,
         0.37113538,  0.65869963, -0.13255566, -0.02887333, -0.83695924,
        -0.22716631, -1.65890098, -2.18556809, -0.18538205, -1.02130592,
        -1.65890098,  1.12810314, -0.89930028,  2.06105924, -1.3888917 ,
        -0.14247988,  0.25021523,  0.        ,  0.93755132, -0.3380031 ]),
 array([ 553, 2020,  603,  115,  383,    6,   98,  316, 2021,  219,  316,
        2953,  152,   23,    5,  266,  514,   52,    5,   21,  196,   50,
           4,   45,  515, 2954,    1,  289,  347, 2955], dtype=int32))

In [ ]:
reshaped_train_y = one_hot_to_categorical(train_y)

In [ ]:
### INPUT KE SMOTE
X_train.shape, Y_train.shape

((2499, 30), (2499,))

In [ ]:
smote_enn = SMOTEENN()
smote_enn_x, smote_enn_y = smote_enn.fit_resample(result, Y_train)

In [ ]:
smote_enn_x.shape, smote_enn_y.shape

((0, 30), (0,))

In [ ]:
### INPUT KE MODEL
train_x.shape, train_y.shape

((2499, 30, 300), (2499, 2))

In [ ]:
res_x_back = np.array(smote_enn_x).reshape(smote_enn_x.shape[0], 30, 300)

In [ ]:
y_matrix = np.zeros((smote_enn_x.shape[0], 2))
for i, line in enumerate(smote_enn_y):
  a = int(smote_enn_y[i])
  if a == 0:
    y_matrix[i][a] = 1
res_y_back = y_matrix

In [ ]:
smote_enn_x.shape

(1673, 30)

In [ ]:
#train model
model_lstm.fit(res_x_back, res_y_back, epochs=100, callbacks=callbacks,
               validation_split=0.1,
               batch_size=32,
               shuffle=False,
               verbose=1)

NameError: name 'model_lstm' is not defined

In [ ]:
evals(model_lstm, test_x, test_y, len(res_x_back), 'Bi-LSTM', 'SMOTE + ENN')

16/16 [==============================] - 0s 11ms/step - loss: 5.1219 - accuracy: 0.5920 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00


In [ ]:
word_vectors.key_to_index["syukur"]

7458

## GRU

In [ ]:
# Create BiLSTM model
def create_bilstm(sentence_length, word2vec_len):
    model = Sequential()
    # Input layer
    model.add(Bidirectional(
              LSTM(64, return_sequences=True),
              input_shape=(sentence_length, word2vec_len)))
    # Hidden layer
    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
#building the gru in keras
def build_gru(sentence_length, word2vec_len):
  model = keras.Sequential()
  model.add(layers.GRU(64, input_shape=(sentence_length, word2vec_len)))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(20))
  model.add(layers.Dense(1, kernel_initializer='normal', activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  return model

In [ ]:
gru_model = build_gru(sentence_length, word2vec_len)

In [ ]:
x_trains = X_train.reshape(-1, 2499, 30)
x_trains.shape

(1, 2499, 30)

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in my_tokenizer.word_index.items():
	embedding_vector = word_vectors.key_to_index[word]
	if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
my_tokenizer = TokenizerTransformer()
my_padder = PadSequencesTransformer(maxlen=30)
my_model = KerasClassifier(
               build_fn=create_model,
               epochs=100,
               embedding_input_dim=vocab_size,
               embedding_output_dim=EMBEDDING_DIM,
               embedding_weights=embedding_weights
)

pipeline = Pipeline([
              ('tokenizer', my_tokenizer),
              ('padder', my_padder),
              ('smote', SMOTE(random_state=42)),
              ('model', my_model)
])
pipeline.fit(data_df['text'], data_df['label'])

In [ ]:
my_tokenizer = TokenizerTransformer()
my_padder = PadSequencesTransformer(maxlen=longest_text)
my_model = KerasClassifier(
  model=create_model,
  epochs=200,
  embedding_input_dim=vocab_size,
  embedding_output_dim=EMBEDDING_DIM,
  embedding_weights=embedding_weights,
  batch_size=1024,
  validation_split=0.1,
  verbose=0)

pipelines = Pipeline([
  ('tokenizer', my_tokenizer),
  ('padder', my_padder),
  ('smote', SMOTE(random_state=42)),
  ('model', my_model)
])
pipelines.fit(data_df['text'], data_df['label'])


In [ ]:
#def evaluate():
test_df = pd.read_csv('/content/increment_datasets_f2/test.txt', sep='\t', header = None)
test_df.columns = ['label', 'text']
y_pred = pipeline.predict(test_df['text'])
acc = accuracy_score(test_df['label'], y_pred)

In [ ]:
acc

In [ ]:
no_smote = Pipeline([
  ('tokenizer', my_tokenizer),
  ('padder', my_padder),
  ('model', my_model)
])
no_smote.fit(data_df['text'], data_df['label'])

In [ ]:
ada_tokenizer = TokenizerTransformer()
ada_padder = PadSequencesTransformer(maxlen=30)
ada_model = KerasClassifier(
               build_fn=create_model,
               epochs=200,
               embedding_input_dim=vocab_size,
               embedding_output_dim=EMBEDDING_DIM,
               embedding_weights=embedding_weights
)

ada = Pipeline([
              ('tokenizer', ada_tokenizer),
              ('padder', ada_padder),
              ('smote', ADASYN()),
              ('model', ada_model)
])
ada.fit(data_df['text'], data_df['label'])

In [ ]:
ada_tokenizer = TokenizerTransformer()
ada_padder = PadSequencesTransformer(maxlen=30)
ada_model = KerasClassifier(
               build_fn=create_model,
               epochs=200,
               embedding_input_dim=vocab_size,
               embedding_output_dim=EMBEDDING_DIM,
               embedding_weights=embedding_weights
)

SMTomek = Pipeline([
              ('tokenizer', ada_tokenizer),
              ('padder', ada_padder),
              ('SMOTETomek', SMOTETomek()),
              ('model', ada_model)
])
SMTomek.fit(data_df['text'], data_df['label'])

In [ ]:
ada_tokenizer = TokenizerTransformer()
ada_padder = PadSequencesTransformer(maxlen=30)
ada_model = KerasClassifier(
               build_fn=create_model,
               epochs=200,
               embedding_input_dim=vocab_size,
               embedding_output_dim=EMBEDDING_DIM,
               embedding_weights=embedding_weights
)

SMENN = Pipeline([
              ('tokenizer', ada_tokenizer),
              ('padder', ada_padder),
              ('SMOTEENN', SMOTEENN()),
              ('model', ada_model)
])
SMENN.fit(data_df['text'], data_df['label'])

In [ ]:
wv = KeyedVectors.load("/content/idwiki_word2vec_300.model", mmap='r')
word_vectorss = wv.wv

In [ ]:
# Make predictions on the test set
predictions = xgb.predict(test_df['text'])
accuracy_score(test_df['label'], predictions)

In [ ]:
y_pred = pipeline.predict(test_df['text'])
accuracy_score(test_df['label'], y_pred)

In [ ]:
y_preds = no_smote.predict(test_df['text'])
accuracy_score(test_df['label'], y_preds)

In [ ]:
y_preds = ada.predict(test_df['text'])
accuracy_score(test_df['label'], y_preds)

In [ ]:
y_pred_tomek = SMTomek.predict(test_df['text'])
accuracy_score(test_df['label'], y_pred_tomek)

In [ ]:
y_pred_smoteenn = SMENN.predict(test_df['text'])
accuracy_score(test_df['label'], y_pred_smoteenn)

In [ ]:
import joblib
joblib.dump(no_smote, '/content/drive/MyDrive/aathesis/save/no_smote.pkl')

In [ ]:
joblib.dump(ada, '/content/drive/MyDrive/aathesis/save/ada.pkl')
joblib.dump(SMTomek, '/content/drive/MyDrive/aathesis/save/SMTomek.pkl')
joblib.dump(SMENN, '/content/drive/MyDrive/aathesis/save/SMENN.pkl')

In [ ]:
pipeline = joblib.load(PIPE_PATH)

In [ ]:
def text_to_df(txt_file):
  data_df = pd.read_csv(txt_file, sep='\t', header = None)
  data_df.columns = ['label', 'text']
  return data_df

In [ ]:
def run_model(train_file, test_file, input_size, percent_dataset):

  #initialize model
  #model = build_cnn(input_size, word2vec_len, num_classes)

  model = xgb.XGBClassifier(n_jobs=-1)

  #load data
  data_df = text_to_df(train_file)
  test_df = text_to_df(test_file)

  my_tokenizer = TokenizerTransformer()
  my_padder = PadSequencesTransformer(maxlen=input_size)
  my_tokenizer.fit(data_df['text'])
  tokenized = my_tokenizer.transform(data_df['text'])
  my_padder.fit(tokenized)
  train_x = my_padder.transform(tokenized)
  train_y = data_df['label']

  test_tokenizer = TokenizerTransformer()
  test_padder = PadSequencesTransformer(maxlen=input_size)
  test_tokenizer.fit(test_df['text'])
  test_tokenized = test_tokenizer.transform(test_df['text'])
  test_padder.fit(test_tokenized)
  test_x = test_padder.transform(test_tokenized)
  test_y = test_df['label']

  #smote = SMOTE()
  #train_x, train_y = smote.fit_resample(X_train, train_y)
  eval_set = [(test_x, test_y)]
  #train model
  model.fit(train_x,
        train_y,
        eval_metric=["error", "logloss"],
        eval_set=eval_set,
        verbose=True)
  #model.save('checkpoints/lol')
  #model = load_model('checkpoints/lol')
  #evaluate model
  y_pred = model.predict(test_x)
  acc = accuracy_score(test_y, y_pred)
  results = model.evals_result()
  #return the accuracy
  #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
  return acc, results

In [ ]:
train_orig = '/content/drive/MyDrive/aathesis/increment_datasets_f2/pc/orig_full.txt'
test_path = '/content/drive/MyDrive/aathesis/increment_datasets_f2/pc/test.txt'

In [ ]:
accs, results= run_model(train_orig, test_path, 50, 1)

In [ ]:
my_tokenizer = TokenizerTransformer()
my_padder = PadSequencesTransformer(maxlen=30)
my_model = KerasClassifier(
  model=create_model,
  epochs=200,
  embedding_input_dim=vocab_size,
  embedding_output_dim=EMBEDDING_DIM,
  embedding_weights=embedding_weights,
  batch_size=1024,
  validation_split=0.1,
  verbose=True)


NameError: name 'create_model' is not defined

In [ ]:
models = build_cnn(30, 300, 2)

In [ ]:
models.fit(train_x, train_y, epochs = 100, verbose = True)

In [ ]:
models.predict(test_x)

In [ ]:
print("Number of unique words in dictionary=",
      len(my_tokenizer.word_index))
print("Dictionary is = ", my_tokenizer.word_index)

Number of unique words in dictionary= 0
Dictionary is =  {}


In [ ]:
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
  voc_size = len(word_index) + 1

  # Adding again 1 because of reserved 0 index
  embedding_matrix_vocab = np.zeros((voc_size,
                                      embedding_dim))

  with open(filepath, encoding="utf8") as f:
    for line in f:
      word, *vector = line.split()
      if word in word_index:
        idx = word_index[word]
        embedding_matrix_vocab[idx] = np.array(
            vector, dtype=np.float32)[:embedding_dim]
  return embedding_matrix_vocab


# matrix for vocab: word_index
embedding_dim = 300
embedding_matrix_vocab = embedding_for_vocab(
    '/content/drive/MyDrive/aathesis/glove.6B.300d.txt', my_tokenizer.word_index,
  embedding_dim)

# print("Dense vector for first word is => ",
#       embedding_matrix_vocab[1])

In [ ]:
embedding_matrix_vocab.shape

(1, 300)

In [ ]:

#get full image paths
def get_txt_paths(folder):
    txt_paths = [join(folder, f) for f in listdir(folder) if isfile(join(folder, f)) and '.txt' in f]
    if join(folder, '.DS_Store') in txt_paths:
        txt_paths.remove(join(folder, '.DS_Store'))
    txt_paths = sorted(txt_paths)
    return txt_paths

#get subfolders
def get_subfolder_paths(folder):
    subfolder_paths = [join(folder, f) for f in listdir(folder) if (isdir(join(folder, f)) and '.DS_Store' not in f)]
    if join(folder, '.DS_Store') in subfolder_paths:
        subfolder_paths.remove(join(folder, '.DS_Store'))
    subfolder_paths = sorted(subfolder_paths)
    return subfolder_paths

#get all image paths
def get_all_txt_paths(master_folder):

    all_paths = []
    subfolders = get_subfolder_paths(master_folder)
    if len(subfolders) > 1:
        for subfolder in subfolders:
            all_paths += get_txt_paths(subfolder)
    else:
        all_paths = get_txt_paths(master_folder)
    return all_paths

In [ ]:
#get the pickle file for the word2vec so you don't have to load the entire huge file each time
def gen_vocab_dicts(folder, huge_word2vec):
  vocab = set()
  #text_embeddings = open(huge_word2vec, 'r').readlines()
  wv = KeyedVectors.load("/content/idwiki_word2vec_300.model", mmap='r')
  text_embeddings = wv.wv
  word2vec = {}

  #get all the vocab
  all_txt_paths = get_all_txt_paths(folder)
  print(all_txt_paths)

  #loop through each text file
  for txt_path in all_txt_paths:

  # get all the words
    try:
      all_lines = open(txt_path, "r").readlines()
      for line in all_lines:
        words = line[:-1].split(' ')
        for word in words:
          vocab.add(word)
    except:
      print(txt_path, "has an error")

    print(len(vocab), "unique words found")

  # load the word embeddings, and only add the word to the dictionary if we need it
  for line in text_embeddings:
    print(line)
    items = line.split(' ')
    word = items[0]
    if word in vocab:
      vec = items[1:]
      word2vec[word] = np.asarray(vec, dtype = 'float32')
  print(len(word2vec), "matches between unique words and word2vec dictionary")
  return word2vec

  # pickle.dump(word2vec, open(output_pickle_path, 'wb'))
  # print("dictionaries outputted to", output_pickle_path)


In [ ]:
wo2vec = gen_vocab_dicts('/content/drive/MyDrive/aathesis',
                        '/content/drive/MyDrive/aathesis/glove.6B.300d.txt')

In [ ]:
w2vec = gen_vocab_dicts('/content/drive/MyDrive/aathesis',
                        '/content/drive/MyDrive/aathesis/glove.6B.300d.txt')

['/content/drive/MyDrive/aathesis/increment_datasets_f2/test.txt']
3025 unique words found
1176 matches between unique words and word2vec dictionary


In [ ]:
len(w2vec)

1176

In [ ]:
def neural_net():
  train_comments = data_df['text']
  train_labels = data_df['label']
  test_comments = test_df['text']
  test_labels = test_df['label']

  max_words = 30
  tokenize = text.Tokenizer(num_words=max_words, char_level=False)
  tokenize.fit_on_texts(train_comments)

  x_tra = tokenize.texts_to_matrix(train_comments)
  x_tes = tokenize.texts_to_matrix(test_comments)

  encoder = LabelEncoder()
  encoder.fit(train_labels)
  y_tra = encoder.transform(train_labels)
  y_tes = encoder.transform(test_labels)
  print(y_tra.shape, y_tes.shape)

  num_classes = np.max(y_tra) + 1
  y_tra = utils.to_categorical(y_tra, num_classes)
  y_tes = utils.to_categorical(y_tes, num_classes)
  print(y_tra.shape)

  batch_size = 32
  epochs = 1
  model = Sequential([
     Embedding(vocab_size, 300, input_length=30),
     Bidirectional(LSTM(64, return_sequences=True)),
     Dropout(0.5),
     Bidirectional(LSTM(32, return_sequences=False)),
     Dropout(0.5),
     Dense(20, activation='relu'),
     Dense(2, activation='sigmoid')
     ])

  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

  history = model.fit(x_tra, y_tra,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_split=0.3)

  score = model.evaluate(x_tes, y_tes,
                       batch_size=batch_size, verbose=1)
  return score[1]

In [ ]:
neural_net()

[0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0.]
(2499,) (500,)
2
(2499, 2) (500, 2)
16/16 [==============================] - 0s 8ms/step - loss: 0.6836 - accuracy: 0.5920


0.5920000076293945

In [ ]:
import pickle

# save the iris classification model as a pickle file
model_pkl_file = "iris_classifier_model.pkl"

with open(model_pkl_file, 'wb') as file:
  pickle.dump(model, file)

In [ ]:
# load model from pickle file
with open(model_pkl_file, 'rb') as file:
  model = pickle.load(file)

In [ ]:
# code for Glove word embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

x = {'text', 'the', 'leader', 'prime',
     'natural', 'language'}

# create the dict.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

# number of unique words in dict.
print("Number of unique words in dictionary=",
      len(tokenizer.word_index))
print("Dictionary is = ", tokenizer.word_index)

# download glove and unzip it in Notebook.
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove*.zip

# vocab: 'the': 1, mapping of words with
# integers in seq. 1,2,3..
# embedding: 1->dense vector
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1

    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix_vocab


# matrix for vocab: word_index
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab(
    '../glove.6B.50d.txt', tokenizer.word_index,
  embedding_dim)

print("Dense vector for first word is => ",
      embedding_matrix_vocab[1])

Number of unique words in dictionary= 6
Dictionary is =  {'text': 1, 'natural': 2, 'leader': 3, 'the': 4, 'language': 5, 'prime': 6}


FileNotFoundError: [Errno 2] No such file or directory: '../glove.6B.50d.txt'